Install packages

In [2]:
!pip install pytdc xgboost rdkit scikit-learn pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta

Import the Dataset

In [16]:
from tdc.single_pred import Tox
from sklearn import svm

data = Tox(name = 'hERG_Karim')
df = data.get_data()
df #view dataframe to explore

Found local copy...
Loading...
Done!


Drug_ID                                               Drug  Y
0            0                  Fc1ccc(-n2cc(NCCN3CCCCC3)nn2)cc1F  1
1            1  COc1cc(N2Cc3ccc(Sc4ccc(F)cc4)nc3C2=O)ccc1OCCN1...  0
2            2  CCOC(=O)[C@H]1CC[C@@H](N2CC(NC(=O)CNc3nn(C(N)=...  0
3            3  N[C@@H](Cn1c(=O)cnc2ccc(F)cc21)C1CCC(NCc2ccc3c...  0
4            4  O=C(NC1COc2cccc(-c3ccnc(CO)c3)c2C1)c1ccc(OCC(F...  0
...        ...                                                ... ..
13440    13440    Cc1csc(NC(=O)c2sc3nc4c(c(C(F)(F)F)c3c2N)CCC4)n1  0
13441    13441              Cc1cccc(-c2n[nH]cc2-c2ccc3ncccc3n2)n1  0
13442    13442   Cc1ccccc1-n1c(Cn2cnc3c(N)ncnc32)nc2cccc(C)c2c1=O  0
13443    13443   Cc1ccccc1-n1c(Cn2ncc3c(N)ncnc32)nc2cccc(C)c2c1=O  0
13444    13444  CCc1c(O)cc(O)c(C(=O)c2ccc(OCCN3CCOCC3)c(OC)c2)...  0

[13445 rows x 3 columns]

Import Necessary Packages and Functions

In [17]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score

from rdkit.Chem import MACCSkeys
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops



Feature Engineering

In [18]:
#data conversion
split = data.get_split()
smiles_train = split['train']['Drug']
Y_train = split['train']['Y']
smiles_valid = split['valid']['Drug']
Y_valid = split['valid']['Y']

# Compute MACCS Keys
def compute_maccs_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    maccs = MACCSkeys.GenMACCSKeys(mol)
    return np.array(maccs)

# Compute additional molecular descriptors
def compute_molecular_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)

    descriptors = []
    # Molecular weight
    descriptors.append(Descriptors.MolWt(mol))#0
    # LogP (octanol-water partition coefficient)
    descriptors.append(Descriptors.MolLogP(mol))#1
    # Topological Polar Surface Area (TPSA)
    descriptors.append(Descriptors.TPSA(mol))#2
    # Number of rotatable bonds
    descriptors.append(Descriptors.NumRotatableBonds(mol))
    # Aromaticity (True/False)
    descriptors.append(Descriptors.NumAromaticRings(mol))
    # Hydrogen Bond Donors and Acceptors
    descriptors.append(Descriptors.NumHDonors(mol))
    descriptors.append(Descriptors.NumHAcceptors(mol))

    return np.array(descriptors)

#Compute Morgan fingerprints
def compute_morgan_fingerprint(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
    return np.array(fingerprint)

# Compute all
def compute_combined_fingerprints(smiles):
    # Compute Morgan Fingerprint
    morgan_fingerprint = compute_morgan_fingerprint(smiles)

    # Compute MACCS Fingerprint
    maccs_fingerprint = compute_maccs_fingerprint(smiles)

    # Compute molecular descriptors
    molecular_descriptors = compute_molecular_descriptors(smiles)

    # Combine all features into a single vector
    combined_features = np.concatenate([morgan_fingerprint, maccs_fingerprint, molecular_descriptors])

    return combined_features

train_features = smiles_train.apply(compute_combined_fingerprints)
X_train_combined = np.stack(train_features.values)

# Compute features for validation data
valid_features = smiles_valid.apply(compute_combined_fingerprints)
X_valid_combined = np.stack(valid_features.values)

# Scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_valid_scaled = scaler.transform(X_valid_combined)


Train and Evaluate Models

In [19]:
# Train Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train_scaled, Y_train)



# Predict and evaluate Random Forest model
y_pred_rf = rf_model.predict(X_valid_scaled)
y_prob_rf = rf_model.predict_proba(X_valid_scaled)
print("Random Forest Validation Accuracy:", accuracy_score(Y_valid, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(Y_valid, y_pred_rf))
print("Random Forest AUC: ", roc_auc_score(Y_valid, y_prob_rf[:,1]))

Random Forest Validation Accuracy: 0.8608630952380952
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.86      0.86       683
           1       0.85      0.86      0.86       661

    accuracy                           0.86      1344
   macro avg       0.86      0.86      0.86      1344
weighted avg       0.86      0.86      0.86      1344

Random Forest AUC:  0.9254512551416174


In [20]:
#Train XGBoost
#Best parameters found from randomized search: {'subsample': 0.8, 'reg_lambda': 3, 'reg_alpha': 1, 'n_estimators': 200, 'max_depth': 9, 'learning_rate': 0.3, 'gamma': 0, 'colsample_bytree': 0.6}
model = XGBClassifier(
    subsample = 0.8,
    reg_lambda = 3,
    reg_alpha = 1,
    max_depth=9,
    learning_rate=0.3,
    n_estimators=200,
    gamma = 0,
    colsample_bytree=0.6,
    random_state=42
)

model.fit(X_train_scaled, Y_train)
y_pred_xg = model.predict(X_valid_scaled)
y_prob_xg = model.predict_proba(X_valid_scaled)
print("XGBoost Validation accuracy:", accuracy_score(Y_valid, y_pred_xg))
print("XGBoost Classification report:\n", classification_report(Y_valid, y_pred_xg))
print("XGBoost AUC: ", roc_auc_score(Y_valid, y_prob_xg[:,1]))

XGBoost Validation accuracy: 0.8586309523809523
XGBoost Classification report:
               precision    recall  f1-score   support

           0       0.87      0.85      0.86       683
           1       0.85      0.87      0.86       661

    accuracy                           0.86      1344
   macro avg       0.86      0.86      0.86      1344
weighted avg       0.86      0.86      0.86      1344

XGBoost AUC:  0.9227511446120724


In [21]:
smiles_test = split['test']['Drug']
Y_test = split['test']['Y']

# Compute features for validation data
valid_features = smiles_test.apply(compute_combined_fingerprints)
X_test_combined = np.stack(valid_features.values)

# Scale the features
X_test_scaled = scaler.transform(X_test_combined)

In [22]:
#predict test for xgboost
y_test_xg = model.predict(X_test_scaled)
yprob_test_xg = model.predict_proba(X_test_scaled)

#predict test for random forest
y_test_rf = rf_model.predict(X_test_scaled)
yprob_test_rf = rf_model.predict_proba(X_test_scaled)

print("XGBoost Test accuracy:", accuracy_score(Y_test, y_test_xg))
print("XGBoost Classification report:\n", classification_report(Y_test, y_test_xg))
print("XGBoost AUC: ", roc_auc_score(Y_test, yprob_test_xg[:,1]))


print("\nRandom Forest Test accuracy:", accuracy_score(Y_test, y_test_rf))
print("Random Forest Classification report:\n", classification_report(Y_test, y_test_rf))
print("Random Forest AUC: ", roc_auc_score(Y_test, yprob_test_rf[:,1]))

XGBoost Test accuracy: 0.8471550762365192
XGBoost Classification report:
               precision    recall  f1-score   support

           0       0.85      0.84      0.85      1346
           1       0.84      0.86      0.85      1343

    accuracy                           0.85      2689
   macro avg       0.85      0.85      0.85      2689
weighted avg       0.85      0.85      0.85      2689

XGBoost AUC:  0.9241361016729748

Random Forest Test accuracy: 0.8549646708813685
Random Forest Classification report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      1346
           1       0.85      0.85      0.85      1343

    accuracy                           0.85      2689
   macro avg       0.85      0.85      0.85      2689
weighted avg       0.85      0.85      0.85      2689

Random Forest AUC:  0.9255686023727677


In [23]:
feature_importances = rf_model.feature_importances_

# Get the names of the features (in this case, combined features like Morgan, MACCS, and molecular descriptors)
# The feature names can be inferred from the fingerprint and descriptor components
morgan_size = len(compute_morgan_fingerprint(smiles_train.iloc[0]))  # Length of Morgan fingerprint
maccs_size = len(compute_maccs_fingerprint(smiles_train.iloc[0]))  # Length of MACCS fingerprint
descriptor_size = len(compute_molecular_descriptors(smiles_train.iloc[0]))  # Number of molecular descriptors

feature_labels = []
for i in range(morgan_size):
    feature_labels.append(f'Morgan_{i}')
for i in range(maccs_size):
    feature_labels.append(f'MACCS_{i}')
for i in range(descriptor_size):
    feature_labels.append(f'Descriptor_{i}')

# Display the feature importances in a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': feature_labels,
    'Importance': feature_importances
})

# Sort the features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the top 15 most important features
print(feature_importance_df.head(15))

           Feature  Importance
1192  Descriptor_1    0.041321
1193  Descriptor_2    0.027608
1191  Descriptor_0    0.018081
1194  Descriptor_3    0.010665
1197  Descriptor_6    0.009701
1196  Descriptor_5    0.009449
807     Morgan_807    0.007353
1170     MACCS_146    0.006719
1160     MACCS_136    0.006148
1195  Descriptor_4    0.006125
893     Morgan_893    0.005590
1178     MACCS_154    0.005182
1155     MACCS_131    0.004954
1164     MACCS_140    0.004897
1163     MACCS_139    0.004179
